<a href="https://colab.research.google.com/github/Jibrangit/DL_Assignments/blob/HW6/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, UpSampling2D

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
import numpy as np
import matplotlib.pyplot as plt
from cv2 import resize
import random

def random_rearrange (X_tr, y_tr, seed):
    np.random.seed(seed)
    np.random.shuffle(X_tr)
    np.random.seed(seed)
    np.random.shuffle(y_tr)

def rechannel(x):
  rechanelled_img = np.reshape(np.repeat(x, 3, axis=1), [48, 48, 3])
  return rechanelled_img
  
y_load = np.load('/content/drive/MyDrive/Colab Notebooks/HW6/ages.npy')
x_loaded = np.load('/content/drive/MyDrive/Colab Notebooks/HW6/faces.npy')
dataset_size = len(x_loaded)

x_load = np.zeros([dataset_size, 48, 48, 3])
for i in range(0, dataset_size):
  x_load[i, :, :, :] = rechannel(x_loaded[i, :, :])

no_data = int(x_load.shape[0])
train_size = int(0.8*0.8*no_data)
test_size = int(0.2*no_data)
print(y_load[0])   
random_rearrange(x_load, y_load, 0)
print(y_load[0])                        #Just checking if shuffling ahs been done coorrectly.
(x_test, x_train, x_valid) = x_load[:test_size], x_load[test_size:test_size+train_size],  x_load[test_size+train_size:]
(y_test, y_train, y_valid) = y_load[:test_size], y_load[test_size:test_size+train_size], y_load[test_size+train_size:]
x_train = x_train.astype('float32') / 255
x_valid = x_valid.astype('float32') / 255
x_test = x_test.astype('float32') / 255

40.0
48.0


In [102]:
model = Sequential()

In [103]:
# first part has two conv64 with a pooling layer
model.add(UpSampling2D(size=(5, 5), input_shape=(48, 48, 3)))
model.build()
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1, activation="relu")) 

In [104]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_16 (UpSampling (None, 240, 240, 3)       0         
_________________________________________________________________
conv2d_156 (Conv2D)          (None, 240, 240, 64)      1792      
_________________________________________________________________
conv2d_157 (Conv2D)          (None, 240, 240, 64)      36928     
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 120, 120, 64)      0         
_________________________________________________________________
conv2d_158 (Conv2D)          (None, 120, 120, 128)     73856     
_________________________________________________________________
conv2d_159 (Conv2D)          (None, 120, 120, 128)     147584    
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 60, 60, 128)     

In [111]:
import keras.backend as K
from keras.metrics import RootMeanSquaredError
def root_mean_squared_error_cust(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-2,
#     decay_steps=10000,
#     decay_rate=0.9)
# opt = keras.optimizers.SGD(learning_rate=lr_schedule)

model.compile(
  loss=root_mean_squared_error_cust,
  optimizer='RMSProp',
  metrics=[RootMeanSquaredError()]
)

In [112]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(x_train,
          y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_valid, y_valid),
          callbacks=[checkpointer])

 68/150 [============>.................] - ETA: 34s - loss: 1769.0529 - root_mean_squared_error: 42.0596

KeyboardInterrupt: ignored